In [ ]:
!cp /content/drive/MyDrive/data/learn_cd4_91K.fa /content
!cp /content/drive/MyDrive/data/learn_cd4_act_91K.txt /content

In [1]:
!cp /content/drive/MyDrive/data/learn_H7-hESC.fa /content
!cp /content/drive/MyDrive/data/learn_H7-hESC_act.txt /content 

In [ ]:
!head learn_cd4.fa

>chr13:19572413-19573013(+)
GTCCCTCCTGGGCAGCTCTTCACACCAGCTCATAGGACCTGGAGCTCACCTGCACTCGGGATTGGGGCAGCCTCCTCCTTCCCTCCGCTTCTCCTCTCCTGGCTCTGAGCATTCACATCTGATTTGGAGAGCTGATAACCTGTTTGCAGAAAACACGTACCTGAATGTTTCGTTCTGTGCTGAGATCTGTGCCTGATCTTCAAGGGCAGGGAGGGGAGGAACACAGCACACATGAAAGGGCAATGCCAGGAAGAGCGCCCTAAATACCAGAGACATGGGAGGCCCCGCCCAGAGCAGGAAGGGCACCTGGTCTTGAGCTCAGAGACTGGAGGGTTGTTTGCTGATGCCCATGCCCAGGGCTAAAGACAGAATGCACggccaggcatggtggcccctgcctgtaactgcagcactttgggagattgtgacttgaggccaggagttcaaggcctgtctgggcaacatacagatcttgtctccaaaagaatagaaaattagttaggcgtagtggtgtatgtctgtagtcccagctactcaggaagctgaggcaggaggatcacctgagcccagaaatttgaggctgcagtgagctgattgtgc
>chr13:19630725-19631325(+)
TCAGAGACCACGTCAGGACTGAGGCCTCCTGCACCAGAACAAGTGTGGACAGGGCCTCAGTGCTGGAGAGGCGAGCCTGGGTCTGGCCACCCACCTCCACAGTCACAGGAGGAGGCCTGGCTGCGGTGCCGCATCTGGGACCATAGGGCAGCCAGGGCAGATGCAGTGCTGAATCCAGAACCCCACGTGTCAGATGGAGACGCTGAGCCTGTCCACCCCACGTACTTCCTCACACTGCCCAGGTCACACCTTCTCCTGGGGCACCAGGTACGGACATCTTAGACTTTTTACATGTGCATGTGAGCACATGTGCACACATCTCTAGTGGACTATGTGCCCATGC

In [2]:
# entend _len is user define length, it is use to lengthen or extend all the sequences upto
# the given input length. As seen above in the fasta file the first line starts with '>' and
# followed by the chromosome and range and in the next line the nucleotides for that range is given.
# so in #1 it just scaning through the whole dataset of the fasta file and determining the 
# longest sequence, in this experiment each is of length 600. 
from collections import OrderedDict

def hash_sequence_1hot(fasta_file,extend_len=None):
  #1 determine longest sequence
  if extend_len is not None:
    seq_len=extend_len
  else:
    seq_len=0
    seq=''
    for line in open(fasta_file):
      if line[0]=='>':
        if seq:
          seq_len=max(seq_len,len(seq))
        seq=''
      else:
        seq+=line.rstrip()
    if seq: #2
      seq_len=max(seq_len,len(seq)) 
  
  seq_vecs=OrderedDict()
  seq=''
  for line in open(fasta_file):
    if line[0]=='>':
      if seq:
        seq_vecs[header]=dna_one_hot(seq,seq_len) 
      header=line[1:].rstrip()
      seq=''
    else:
      seq+=line.rstrip()
  if seq:
    seq_vecs[header]=dna_one_hot(seq,seq_len)
  return seq_vecs
  

In [ ]:
'''seq_vector=hash_sequence_1hot('/content/learn_cd4.fa')'''

In [3]:
# Depending on the neucleotide a value is assigned in the matrix. if some other alphabet occurs than
# it will be set into an random position between 0 to 3 ( 4 location in the matrix for ATGC).
# if n_randon is False then the matrix data type will be float16 in that case for an unknown
# alphabet 1/4 that is 0.25 is assigned ( all ATGC have the same probability). But in case 
# of bool, True is assigned randomly, it depends on the design of your experiment.
# Finally flatten the matrix into 1D matrix of size 4N x 1. 
import random
import numpy as np

def dna_one_hot(seq,seq_len=None,flatten=True,n_random=True):
  if seq_len==None:
    seq_len=len(seq)
    seq_start=0
  else:
    if seq_len<=len(seq):
      # trim the sequence
      seq_trim=(len(seq)-seq_len)//2
      seq=seq[seq_trim:seq_trim+seq_len]
      seq_start=0
    else:
      seq_start=(seq_len-len(seq))//2
  
  seq=seq.upper()

  if n_random:
    seq_code=np.zeros((seq_len,4),dtype='bool')
  else:
    seq_code=np.zeros((seq_len,4),dtype='float16')

  for i in range(seq_len):
    if i>=seq_start and i-seq_start<len(seq):
      nt=seq[i-seq_start] # nt means neucleotide
      if nt=='A':
        seq_code[i,0]=1
      elif nt=='C':
        seq_code[i,1]=1
      elif nt=='G':
        seq_code[i,2]=1
      elif nt=='T':
        seq_code[i,3]=1
      else:
        if n_random:
          ni=random.randint(0,3)
          seq_code[i,ni]=1
        else:
          seq_code[i,:]=0.25
  if flatten:
    seq_vec=seq_code.flatten()[None,:]# instead of None, -1 is also given?
  return seq_vec

In [ ]:
'''bools=seq_vector['chr13:97877063-97877663(+)']
a=bools.shape
for i in range(0,a[1]):print(bools[0][i],end=' ')'''

In [4]:
# The input to this function is an activity file.
import sys
import numpy as np

def hash_scores(scores_file):

  seq_scores={}
  for line in open(scores_file): #1
    a=line.split()
    try:
      seq_scores[a[0]]=np.array([float(a[i]) for i in range(1,len(a))])
    except:
      print('Ignoring header',file=sys.stderr)
  int_scores=True
  for header in seq_scores: #2
    if not np.equal(np.mod(seq_scores[header],1),0).all():
      int_scores=False
      break

  if int_scores: #3
    for header in seq_scores:
      seq_scores[header]=seq_scores[header].astype('int8')

  return seq_scores


In [5]:
# In this function the sequences and the scores are aligned. In this example out of 164
# cells type each range contribute to some of these types, and these information is stored
# in the scores as 1D array consisting of 0 and 1 (in this case) 1 means it contributing 0 
# means not contributing to that cell type?. At first sort the keys of the dictionary so 
# that it become easy to align.
# In this example the dimension of the sequence
# is No_of_sequences X (600x4) times four coz each is coded into 4 True False channels. 
# there is a totol of 600 base pair in each of these rows. The train_scores size will be
# No_of_sequences X 164. The 164 int8 will be filled with either 0 or 1.
# #2(below) whether true of false one thing to notice is that it may happen that the activity
# file header may nor get parsed properly sometimes, so it is good that seq_headers is
# populated with the keys from the seq_vecs rather than seq_scores. 
# This is a bug in the hash_score program. 

def align_seqs_scores_1hot(seq_vecs,seq_scores,sort=True):
  if sort:
    seq_headers=sorted(seq_vecs.keys())
  else:
    seq_headers=seq_vecs.keys() #2

  train_scores=[]
  train_seqs=[]
  for header in seq_headers: #1
    train_seqs.append(seq_vecs[header])
    train_scores.append(seq_scores[header])
  
  train_seqs=np.vstack(train_seqs)  
  train_scores=np.vstack(train_scores)

  return train_seqs,train_scores

In [6]:
import numpy.random as npr

def load_data_1hot(fasta_file,score_file,extend_len=None,mean_norm=True,whiten=False,permute=True,sort=False):
  seq_vecs=hash_sequence_1hot(fasta_file,extend_len)
  seq_scores=hash_scores(score_file)
  train_seqs,train_scores=align_seqs_scores_1hot(seq_vecs,seq_scores,sort)
  
  if whiten:
    #train_scores=preprocessing.scale(train_scores)
    print("white")
  elif mean_norm:
    train_scores-=np.mean(train_scores,axis=0)

  if permute:
    order=npr.permutation(train_seqs.shape[0])
    train_seqs=train_seqs[order]
    train_scores=train_scores[order]

  return train_seqs,train_scores

In [ ]:
seqs,targets=load_data_1hot('/content/learn_cd4_mult.fa','/content/learn_cd4_act_mult.txt',
                    extend_len=None, mean_norm=False, whiten=False, permute=False, sort=False)

Ignoring header


In [7]:
# takes around 3 minutes
seqs,targets=load_data_1hot('/content/learn_H7-hESC.fa','/content/learn_H7-hESC_act.txt',
                    extend_len=None, mean_norm=False, whiten=False, permute=False, sort=False)

In [ ]:
targets.shape,seqs.shape

((47387, 1), (47387, 2400))

In [8]:
targets.shape,seqs.shape

((517655, 1), (517655, 2400))

In [ ]:
targets.shape,seqs.shape

((47605, 165), (47605, 2400))

In [ ]:
#91K
targets.shape,seqs.shape

((91544, 1), (91544, 2400))

In [9]:
# give any file name .pkl is not a real file extension just given by me, also
# 'wb' means write binary. otherwise utf-8?
import pickle
pickle.dump(seqs,open('/content/seqs_91K.pkl','wb'))
pickle.dump(targets,open('/content/targets_91K.pkl','wb'))

In [10]:
!cp /content/seqs_91K.pkl /content/drive/MyDrive/data
!cp /content/targets_91K.pkl /content/drive/MyDrive/data

In [ ]:
targets[400]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int8)